In [1]:
import pandas as pd
import numpy as np
from pyesef.const import PATH_PROJECT_ROOT, CSV_SEPARATOR

In [2]:
def char_len(x, fixed_n):
    '''set string x to fixed_n character, prepend with 'xxx' if short'''
    if isinstance(x, float):
        return x

    if len(x) > fixed_n: 
        return x[:fixed_n] 
    elif len(x) < fixed_n: 
        return ' ' * (fixed_n - len(x)) + x 
    return x
 

### Load and cleanup data

In [5]:

# NB: if this is not working, check that CSV-file is in a tidy format
df = pd.read_csv(f"{PATH_PROJECT_ROOT}/output.csv", sep=CSV_SEPARATOR)
df['legal_name'] = df['legal_name'].apply(lambda x: char_len(x, 40))

# Remove non-unique values
df = df.drop_duplicates(subset=df.columns)

# Tidy up formatting of values
df["value"] = df["value"] / 1e6

In [6]:
filtered_df = df.query(
    'period_end in ("2021-12-31")'
    #' & statement_item_group in ("Revenue", "CashAndCashEquivalents", "CostOfSales", "Assets")'
    #' & lei=="549300MGVITW8GYJHZ50"'
    ' & is_total == False'
    #' & has_resolved_group == False'
    ' & is_extension == False'
    ' & statement_type != "other_comprehensive_income"'
)

DROP_COLS = [
    'has_resolved_group',
    'is_extension',
    "is_total",
    "membership",
    "currency",
]

filtered_df = filtered_df.drop(columns=DROP_COLS)
filtered_df

,period_end,statement_type,statement_item_group,xml_name,xml_name_parent,label,value,lei,legal_name
0,2021-12-31,income_statement,Revenue,RevenueFromContractsWithCustomers,GrossProfit,Revenue from contracts with customers,118.295,549300J9WWQ5CBYQ1M77,Oncopeptides AB.
2,2021-12-31,income_statement,CostOfSales,CostOfSales,GrossProfit,Cost of sales,-53.121,549300J9WWQ5CBYQ1M77,Oncopeptides AB.
6,2021-12-31,income_statement,ResearchAndDevelopmentExpense,ResearchAndDevelopmentExpense,OperatingExpense,Research and development expense,-679.926,549300J9WWQ5CBYQ1M77,Oncopeptides AB.
8,2021-12-31,income_statement,SalesAndMarketingExpense,SalesAndMarketingExpense,OperatingExpense,Sales and marketing expense,-698.346,549300J9WWQ5CBYQ1M77,Oncopeptides AB.
10,2021-12-31,income_statement,SellingGeneralAndAdministrativeExpense,AdministrativeExpense,OperatingExpense,Administrative expenses,-175.459,549300J9WWQ5CBYQ1M77,Oncopeptides AB.
...,...,...,...,...,...,...,...,...,...
24922,2021-12-31,cash_flow_statement,NaN,EffectOfExchangeRateChangesOnCashAndCashEquiva...,NaN,Effect of exchange rate changes on cash and ca...,-360.000,5299008ZUAXN43LVZF54,Sandvik AB
24924,2021-12-31,balance_sheet,CashAndCashEquivalents,CashAndCashEquivalents,CurrentAssets,Cash and cash equivalents,-13585.000,5299008ZUAXN43LVZF54,Sandvik AB
24926,2021-12-31,cash_flow_statement,NaN,CashFlowsFromUsedInOperatingActivitiesContinui...,IncreaseDecreaseInCashAndCashEquivalents,Cash flows from (used in) operating activities...,-13217.000,5299008ZUAXN43LVZF54,Sandvik AB
24928,2021-12-31,cash_flow_statement,NaN,CashFlowsFromUsedInInvestingActivitiesContinui...,IncreaseDecreaseInCashAndCashEquivalents,Cash flows from (used in) investing activities...,26192.000,5299008ZUAXN43LVZF54,Sandvik AB


In [8]:
table = pd.pivot_table(
    filtered_df,
    values='value',
    #index=['lei', "legal_name", "period_end", "statement_type", "statement_item_group", "xml_name"],
    index=['lei', "legal_name", "period_end",],
    columns=['statement_item_group'],
    aggfunc=np.sum
).reset_index()

table

statement_item_group,lei,legal_name,period_end,CashAndCashEquivalents,CostOfSales,CurrentPayables,CurrentTaxes,GainsLosses,IntangibleAssets,IntangibleAssetsOtherThanGoodwill,NetFinanceIncomeCost,PropertyPlantAndEquipment,ResearchAndDevelopmentExpense,Revenue,SalesAndMarketingExpense,SellingGeneralAndAdministrativeExpense,TotalEquity
0,2138001H6FCSZBP26351,ITAB Shop Concept AB,2021-12-31,-208.000,-4565.000,971.000,-53.000,130.00,-1644.000,-112.000,-60.000,-17.000,NaN,6285.000,-1101.000,-395.000,2782.000
1,2138006PZH76JOS6MN27,Kinnevik AB,2021-12-31,-10544.000,NaN,NaN,-5.000,NaN,NaN,NaN,136.000,-6.000,NaN,10.000,NaN,-322.000,NaN
2,2138008KECUUB9VR4798,Ratos AB,2021-12-31,-2230.000,-13448.000,2430.000,-166.000,332.00,-10028.000,NaN,-350.000,-5006.000,NaN,22551.000,NaN,-6695.000,13327.000
3,2138009N2FO2BZTVYS76,Eastnine AB (publ),2021-12-31,-29.201,NaN,NaN,NaN,NaN,NaN,-0.002,NaN,-1.353,NaN,21.530,NaN,-3.853,NaN
4,213800CGP4AKXSLWWS08,DistIT AB,2021-12-31,-35.642,-1962.292,NaN,-14.686,NaN,-374.377,-60.599,-17.549,-36.778,NaN,2524.256,NaN,-448.419,601.177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,549300ZHO4JCQQI13M69,Saab AB (publ),2021-12-31,-1701.000,-30949.000,3534.000,-552.000,-210.00,NaN,NaN,-723.000,-2472.000,-1659.0,39300.000,-2266.000,-1510.000,273.000
59,549300ZUNJ5HNG264Q34,Malmbergs Elektriska AB (publ),2021-12-31,-72.959,-342.796,50.056,-9.045,NaN,-5.402,NaN,-0.311,-102.226,NaN,636.867,NaN,-108.029,NaN
60,894500JU9WRAJQOVBI12,AB SKF,2021-12-31,-13219.000,-58457.000,9881.000,-2484.000,NaN,-10924.000,-6018.000,-695.000,-2661.000,-2751.0,81732.000,-9736.000,-514.000,45365.000
61,984500AFAA591N80BA13,FNG Finance Belgium BV,2021-12-31,-196.302,-1745.602,421.064,-14.663,111.88,-677.039,NaN,-178.713,-0.051,NaN,3625.250,-1406.615,-429.456,NaN


In [9]:
# Ratios
table["ratio_gross_margin"] = 1 - (table["Revenue"] + table["CostOfSales"]) / table["Revenue"]
table["ratio_sga_revenue"] = (-table["SellingGeneralAndAdministrativeExpense"] / table["Revenue"])
#table["cash_to_ta"] = table["CashAndCashEquivalents"] / table["Assets"]

table
#t = table.sort_values(by=['statement_type', "statement_item_group"], ascending=False, na_position='last')
#print(t.to_markdown())

statement_item_group,lei,legal_name,period_end,CashAndCashEquivalents,CostOfSales,CurrentPayables,CurrentTaxes,GainsLosses,IntangibleAssets,IntangibleAssetsOtherThanGoodwill,NetFinanceIncomeCost,PropertyPlantAndEquipment,ResearchAndDevelopmentExpense,Revenue,SalesAndMarketingExpense,SellingGeneralAndAdministrativeExpense,TotalEquity,ratio_gross_margin,ratio_sga_revenue
0,2138001H6FCSZBP26351,ITAB Shop Concept AB,2021-12-31,-208.000,-4565.000,971.000,-53.000,130.00,-1644.000,-112.000,-60.000,-17.000,NaN,6285.000,-1101.000,-395.000,2782.000,0.726333,0.062848
1,2138006PZH76JOS6MN27,Kinnevik AB,2021-12-31,-10544.000,NaN,NaN,-5.000,NaN,NaN,NaN,136.000,-6.000,NaN,10.000,NaN,-322.000,NaN,NaN,32.200000
2,2138008KECUUB9VR4798,Ratos AB,2021-12-31,-2230.000,-13448.000,2430.000,-166.000,332.00,-10028.000,NaN,-350.000,-5006.000,NaN,22551.000,NaN,-6695.000,13327.000,0.596337,0.296883
3,2138009N2FO2BZTVYS76,Eastnine AB (publ),2021-12-31,-29.201,NaN,NaN,NaN,NaN,NaN,-0.002,NaN,-1.353,NaN,21.530,NaN,-3.853,NaN,NaN,0.178960
4,213800CGP4AKXSLWWS08,DistIT AB,2021-12-31,-35.642,-1962.292,NaN,-14.686,NaN,-374.377,-60.599,-17.549,-36.778,NaN,2524.256,NaN,-448.419,601.177,0.777374,0.177644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,549300ZHO4JCQQI13M69,Saab AB (publ),2021-12-31,-1701.000,-30949.000,3534.000,-552.000,-210.00,NaN,NaN,-723.000,-2472.000,-1659.0,39300.000,-2266.000,-1510.000,273.000,0.787506,0.038422
59,549300ZUNJ5HNG264Q34,Malmbergs Elektriska AB (publ),2021-12-31,-72.959,-342.796,50.056,-9.045,NaN,-5.402,NaN,-0.311,-102.226,NaN,636.867,NaN,-108.029,NaN,0.538254,0.169626
60,894500JU9WRAJQOVBI12,AB SKF,2021-12-31,-13219.000,-58457.000,9881.000,-2484.000,NaN,-10924.000,-6018.000,-695.000,-2661.000,-2751.0,81732.000,-9736.000,-514.000,45365.000,0.715228,0.006289
61,984500AFAA591N80BA13,FNG Finance Belgium BV,2021-12-31,-196.302,-1745.602,421.064,-14.663,111.88,-677.039,NaN,-178.713,-0.051,NaN,3625.250,-1406.615,-429.456,NaN,0.481512,0.118462


### Analyse count per item name

In [13]:
a = filtered_df.groupby(["xml_name", "statement_type"])["lei"].count().reset_index(name='count').sort_values(['xml_name'], ascending=False)
print(a.to_markdown())

|     | xml_name                                                                                                                                                                                                     | statement_type                                                                                                                                                                                                          |   count |
|----:|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------:|
| 851 | WorkInProgress                                                                                            